In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from ast import literal_eval

In [74]:
file_names = glob.glob("Zip Code Data (Raw)/*.csv")
"""
    We assume that these key words do not imply that the store is a cafe,
    rather just a place that serves coffee.
        Fast Food,
        Convenience Stores
        Gas station
        Home services
"""
filterwords = {'burgers', 'fast', 'convenience', 'gas', 'bars', 'nightlife', 'delis', 'home'}

In [75]:
def open_filter_files (files):
    """
        Given a list of files, return a dataframe, 
        where each column is a zip code, 
        and each row is filtered by some key words
    """
    
    # Create a dataframe for websites, and a dataframe for key words.
    keywords = pd.DataFrame ()
    websites = pd.DataFrame ()
    for file in files:
        csv = pd.read_csv (file)
        name = file.strip('Zip Code Data (Raw)/').strip('.csv')
        col = [
            'category-str-list', 'category-str-list 2',
            'category-str-list 3', 'category-str-list 4']
        cat = csv [col].replace(np.nan, '', regex=True)
        
        cat[name] = cat [col[0]].astype(str) + ' ' + cat [col[1]] + ' ' + cat [col[2]] + ' ' + cat [col[3]]
        cat[name] = cat[name].apply (lambda x : re.sub('[&,()]', '', x))
        cat[name] = cat[name].str.lower()
        keywords = pd.concat ([keywords, cat [name]], axis=1)
        
        csv = csv.rename(columns = {'js-analytics-click href': name})
        websites = pd.concat ([websites, csv [name]], axis=1)
        
    websites = websites.replace(np.nan, '', regex=True)
    keywords = keywords.replace(np.nan, "[]", regex=True)
    
    # Filter the websites based on key words
    cols = websites.columns
    filtered = pd.DataFrame ()
    for zip_code in cols:
        webs = websites[zip_code]
        words = keywords[zip_code]
        x = webs [words.apply (lambda x: len (set(x.split()).intersection (filterwords)) == 0)]
        filtered = pd.concat ([filtered, x], axis=1)
    return filtered
websites = open_filter_files (file_names)
websites.head()

,60004,60005,60006,60637
0,https://www.yelp.com/biz/around-cafe-arlington...,https://www.yelp.com/biz/around-cafe-arlington...,https://www.yelp.com/biz/around-cafe-arlington...,https://www.yelp.com/biz/plein-air-cafe-and-ea...
1,https://www.yelp.com/biz/coquelicot-gallery-an...,https://www.yelp.com/biz/uptown-cafe-arlington...,https://www.yelp.com/biz/uptown-cafe-arlington...,https://www.yelp.com/biz/robust-coffee-lounge-...
2,https://www.yelp.com/biz/uptown-cafe-arlington...,https://www.yelp.com/biz/coquelicot-gallery-an...,https://www.yelp.com/biz/the-bakester-arlingto...,https://www.yelp.com/biz/harper-cafe-chicago?o...
3,https://www.yelp.com/biz/bread-and-butter-cafe...,https://www.yelp.com/biz/releaf-matcha-caf%C3%...,https://www.yelp.com/biz/bangkok-caf%C3%A9-arl...,https://www.yelp.com/biz/build-coffee-chicago?...
4,NaN,https://www.yelp.com/biz/jelly-cafe-mount-pros...,https://www.yelp.com/biz/starbucks-arlington-h...,https://www.yelp.com/biz/sanctuary-cafe-chicag...


In [76]:
def series_of_all_websites (df):
    websites = pd.DataFrame()
    for col in df.columns:
        x = df[col].replace('', np.nan)
        x = x.dropna()
        websites = pd.concat ([websites, x], ignore_index=True)
    websites = websites.drop_duplicates().reset_index().drop (['index'], axis=1)
    return websites

trial = series_of_all_websites (websites)

In [77]:
trial#.reset_index().drop (['index'], axis=1)#[:300]

,0
0,https://www.yelp.com/biz/around-cafe-arlington...
1,https://www.yelp.com/biz/coquelicot-gallery-an...
2,https://www.yelp.com/biz/uptown-cafe-arlington...
3,https://www.yelp.com/biz/bread-and-butter-cafe...
4,https://www.yelp.com/biz/cafe-de-oro-riverwood...
5,https://www.yelp.com/biz/the-bakester-arlingto...
6,https://www.yelp.com/biz/honey-jam-cafe-arling...
7,https://www.yelp.com/biz/jelly-cafe-mount-pros...
8,https://www.yelp.com/biz/morning-blend-mount-p...
9,https://www.yelp.com/biz/releaf-matcha-caf%C3%...


In [23]:
#websites